In [ ]:
""" Script to implement IntraVoxel Incoherent Motion (IVIM) Analysis."""

import numpy as np
from scipy.optimize import curve_fit, nnls
from scipy.signal import find_peaks
from tqdm import tqdm

In [ ]:
def nnlsfit(data, bvals, region=None):
    """ Implement Non Negative Fitting for IVIM

    Parameters
    ----------
    data: array-like, 4D Matrix
    bvals: array-like
    region: array-like, [(x1,x2),(y1,y2),(z1,z2)]
        optional (default= None)

    Returns
    -------
    d_star: 3D Matrix
        The Pseudo Diffusion Coefficient.
    d: 3D Matrix
        The Diffusion Coefficient.
    pfraction: 3D Matrix
        The Perfusion Fraction.

    """

    # verify the region and set the bounds of iteration
    if region is None:
        x1, x2 = 0, data.shape[0]-1
        y1, y2 = 0, data.shape[1]-1
        z1, z2 = 0, data.shape[2]-1
    else:
        x1, x2 = region[0][0], region[0][1]
        y1, y2 = region[1][0], region[1][1]
        z1, z2 = region[2][0], region[2][1]

    # prepare for NNLS
    NUMEL_D = 400
    DIFS = 10**(np.linspace(-3, 3, NUMEL_D))/1000
    M = np.zeros([len(bvals), 400])
    for i, bval in enumerate(bvals):
        for j, dif in enumerate(DIFS):
            M[i, j] = np.exp(-bval*dif)

    # Create Bar of progress and alocate memory
    MAXCOUNT = (x2-x1)*(y2-y1)*(z2-z1)
    bar = tqdm(total=MAXCOUNT, position=0)

    s = np.zeros(data.shape[3])
    d_star = np.zeros(data.shape[0:3])
    d = np.zeros(data.shape[0:3])
    f = np.zeros(data.shape[0:3])
    
    # NNLS Fitting
    for i in range(x1, x2):

        for j in range(y1, y2):

            for k in range(z1, z2):

                # Fit
                if data[i, j, k, 0] != 0:
                    s = data[i, j, k, :]/data[i, j, k, 0]
                    f1, f2 = 0, 0
                    sk = nnls(M, s)
                    sk[0][0:25] = 0
                    sk[0][375:400] = 0

                    pks, _ = find_peaks(np.concatenate([[min(sk[0])], sk[0], [min(sk[0])]])
                                        )
                    pks = pks-1

                    for _, pk in enumerate(pks):

                        sort = sorted(sk[0][pks], reverse=True)

                        if sk[0][pk] == np.amax(sk[0][pks]):
                            d[i, j, k] = DIFS[pk]
                            f1 = sk[0][pk]

                        elif len(sort) >= 2 and sk[0][pk] == sort[1]:
                            d_star[i, j, k] = DIFS[pk]
                            f2 = np.sum(sk[0][pk-25:pk+25])
                    if (f1+f2) == 0:
                        f[i,j,k] = 0
                    else:
                        f[i, j, k] = f2/(f1 +f2)

                bar.update()

    bar.close()

    return d_star, d, f

In [ ]:
def func(bval, d, d_star, pf):
    return (1 - pf)*np.exp(- bval * d) + pf*np.exp(- bval * d_star)


def nonlinearfit(data, bvals, region=None):
    """ Implement Non Linear Fitting for IVIM

    Parameters
    ----------
    data: array-like, 4D Matrix
    bvals: array-like
    region: array-like, [(x1,x2),(y1,y2),(z1,z2)]
        optional (default= None)

    Returns
    -------
    d_star: 3D Matrix
        The Pseudo Diffusion Coefficient.
    d: 3D Matrix
        The Diffusion Coefficient.
    pfraction: 3D Matrix
        The Perfusion Fraction.

    """

    # find the initial conditions with nnls fit
    d_star0, d0, pfraction0 = nnlsfit(data, bvals, region)

    # verify the region and set the bounds of iteration
    if region is None:
        x1, x2 = 0, data.shape[0]-1
        y1, y2 = 0, data.shape[1]-1
        z1, z2 = 0, data.shape[2]-1
    else:
        x1, x2 = region[0][0], region[0][1]
        y1, y2 = region[1][0], region[1][1]
        z1, z2 = region[2][0], region[2][1]

    # Create Bar of progress and alocate memory
    MAXCOUNT = (x2-x1)*(y2-y1)*(z2-z1)
    
    bar = tqdm(total=MAXCOUNT, position=0)

    s = np.zeros(data.shape[3])
    d_star = np.zeros(data.shape[0:3])
    d = np.zeros(data.shape[0:3])
    pfraction = np.zeros(data.shape[0:3])

    # Non Linear Least Squares Fitting
    for i in range(x1, x2):

        for j in range(y1, y2):

            for k in range(z1, z2):

                if data[i, j, k, 0] != 0:
                    while True:
                        s = data[i, j, k, :] / data[i, j, k, 0]
                        p0 = [d0[i, j, k], d_star0[i, j, k], pfraction0[i, j, k]]
                        try:
                            popt = curve_fit(func, bvals, s, p0=p0, bounds=(0,np.inf))
                        except RuntimeError:
                            while True:
                                p0 = [0.001,0.015,0.2]
                                try:
                                    popt = curve_fit(func, bvals, s, p0=p0,  bounds=(0,np.inf))
                                except RuntimeError:
                                    popt = [ [d0[i, j, k], d_star0[i, j, k], pfraction0[i, j, k]] , [[0,0],[0,0]] ]
                                    break
                                break
                        break
                    d[i, j, k] = popt[0][0]
                    d_star[i, j, k] = popt[0][1]
                    pfraction[i, j, k] = popt[0][2]   
                bar.update()

    bar.close()

    return d_star, d, pfraction
